# Introduction

Provided the data downloaded from www.euets.info as zip file, we extract the data using the *ziploader* sub-module:

# Packages and options

In [11]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyeutl.ziploader import (get_installations, get_accounts, get_transactions) 

In [12]:
fn_zip = "./eutl.zip"

# Get Data

Get installation data from zip file. The function merges information from lookup tables into the installation table:

In [5]:
df_inst = get_installations(fn_zip, )
df_inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17644 entries, 0 to 17643
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   17644 non-null  object 
 1   name                 16057 non-null  object 
 2   registry_id          17644 non-null  object 
 3   activity_id          17644 non-null  int64  
 4   eprtrID              5366 non-null   object 
 5   parentCompany        6178 non-null   object 
 6   subsidiaryCompany    3562 non-null   object 
 7   permitID             16057 non-null  object 
 8   designatorICAO       603 non-null    object 
 9   monitoringID         1587 non-null   object 
 10  monitoringExpiry     0 non-null      float64
 11  monitoringFirstYear  1586 non-null   object 
 12  permitDateExpiry     4488 non-null   object 
 13  isAircraftOperator   17644 non-null  bool   
 14  ec748_2009Code       1587 non-null   object 
 15  permitDateEntry      16047 non-null 

Next, load the account data. The account dataframe includes the information of related look-up tables. If a dataframe with installation information is passed to the function, the installation information is also merged into the account table:

In [6]:
df_acc = get_accounts(fn_zip, 
                          df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]]) 
df_acc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42007 entries, 0 to 42006
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         42007 non-null  int64  
 1   name                       41310 non-null  object 
 2   registry_id                41290 non-null  object 
 3   accountHolder_id           41284 non-null  float64
 4   accountType_id             41283 non-null  object 
 5   isOpen                     41310 non-null  object 
 6   openingDate                41254 non-null  object 
 7   closingDate                24498 non-null  object 
 8   commitmentPeriod           19192 non-null  object 
 9   companyRegistrationNumber  36967 non-null  object 
 10  isRegisteredEutl           42007 non-null  bool   
 11  installation_id            30501 non-null  object 
 12  registry                   42007 non-null  object 
 13  accountType                41283 non-null  obj

Transactions follow the same logic also providing the descriptions provided in the look-up tables. In addition, we can pass a table with account information. The transaction loader also allows to resample the transaction data to a higher frequency: 

In [8]:
freq = "M"
# get transactions merged with account information
cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
        'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
        'accountCategory', 'installationName', 'activity_id', 'activity', 
        'nace_id', 'nace', "naceCategory" ]
df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                      df_account=df_acc[cols])
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6068105 entries, 0 to 6068104
Data columns (total 42 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   date                             datetime64[ns]
 1   id                               int64         
 2   transactionID                    object        
 3   transactionTypeMain_id           int64         
 4   transactionTypeSupplementary_id  int64         
 5   transferringAccount_id           float64       
 6   acquiringAccount_id              float64       
 7   unitType_id                      object        
 8   amount                           int64         
 9   transactionTypeMain              object        
 10  transactionTypeSupplementary     object        
 11  unitType                         object        
 12  transferringName                 object        
 13  transferringRegistry_id          object        
 14  transferringAccountHolder_id     f

We can wrap all of this into a single function, e.g.:

In [10]:
def load_data(fn_zip, freq="D"):
    """Load installation account and transaction data from zip file
    :param fn_zip: <string> name of zip file with eutl data
    :param freq: <string> frequency for resampling of transaction data"""
    # get installations
    df_inst = get_installations(fn_zip)
    # get accounts with selected installation information
    df_acc = get_accounts(fn_zip, 
                          df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]]) 
    # get transactions merged with account information
    cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
            'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
            'accountCategory', 'installationName', 'activity_id', 'activity', 
            'nace_id', 'nace', "naceCategory" ]
    df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                          df_account=df_acc[cols])
    return df_inst, df_acc, df_trans

#df_inst, df_acc, df_trans = load_data(fn_zip, freq="M")